## 單篇文章爬取結果

In [161]:
from bs4 import BeautifulSoup, NavigableString
from datetime import datetime
import requests, re

start_time = time.time()

# 連結網站
response = requests.get("https://tfc-taiwan.org.tw/articles/5665")
 
# HTML原始碼解析
soup = BeautifulSoup(response.text, "html.parser")

# 製作爬取兩個tag間文字的function
def between(cur, end):
    while cur and cur != end:
        if isinstance(cur, NavigableString):
            text = cur.strip()
            if len(text):
                yield text
        cur = cur.next_element

# 輸出訊息文本（介於'背景'與'查核'之間的文字）
# tem_article = ' '.join(text for text in between(soup.find('h2', text='背景').next_sibling,soup.find('h2', text='查核')))
# article = re.search('「(.*)」', tem_article)
# print(article.group(1))

# 輸出訊息文本（介於'背景'與'查核'之間的文字）without .next_sibling
tem_article = ' '.join(text for text in between(soup.find('h2', text=re.compile('背景')),soup.find('h2', text=re.compile('查核'))))
if len(tem_article)!=0:
    article = re.search('「(.*)」', tem_article)
    if article != None:
        print(article.group(1))
    elif re.search('稱：(.*)圖1',tem_article) != None:
        article2 = re.search('稱：(.*)圖1',tem_article)
        print(article2.group(1))
else:
    tem_article2 = ' '.join(text for text in between(soup.find('p', text=re.compile('背景')),soup.find('h2', text=re.compile('查核'))))
    article = re.search('：(.*)圖1', tem_article2)
    try:
        print(article.group(1))
    except AttributeError:
        print("AttributeError")
    

# 取得查核結果與主題分類
node_tags = soup.find_all("div", {"class": "node-tags"})
node_tag = node_tags[0].get_text()
node_tag_filter = list(filter(None, node_tag.split("\n")))
print(node_tag_filter[1]) # 查核結果
print(node_tag_filter[2]) # 主題分類

# 取得更新日期
submitted = soup.find_all("div", {"class": "submitted"})
date = datetime.strptime(submitted[0].get_text(), "%Y-%m-%d").date()
print(date)

print("--- %s seconds ---" % (time.time() - start_time))

AttributeError
事實釐清
健康
2021-05-31
--- 0.538902997970581 seconds ---


In [135]:
soup.find('h2', text=re.compile('背景'))

## 爬取單一頁面10筆文章的連結

In [231]:
response = requests.get("https://tfc-taiwan.org.tw/articles/report") # 事實查核報告分類第一頁，有十筆文章可點選

soup = BeautifulSoup(response.content, "lxml") # 解析HTML原始碼
a_tags = soup.find_all("div", {"class": "view-content"}) # 包含該頁10篇文章連結的區塊
herfs = a_tags[0].find_all("a",attrs={'href': re.compile("^/articles/")}) # 取得區塊內的所有連結

links = []
for link in herfs:
    if "category" in link.get('href') or link.get('href') in links: # 只保留事實查核報告的連結，且連結不重複
        continue
    links.append(link.get('href'))
print(links)

['/articles/6665', '/articles/6664', '/articles/6660', '/articles/6659', '/articles/6657', '/articles/6656', '/articles/6654', '/articles/6652', '/articles/6650', '/articles/6649']


## 爬取所有文章連結

In [3]:
pip install grequests

Note: you may need to restart the kernel to use updated packages.


In [1]:
from bs4 import BeautifulSoup, NavigableString
import requests, re, grequests

# 製作事實查核報告列表1-137頁的網址清單（list）
page_w_10 = []
page_w_10 = ["https://tfc-taiwan.org.tw/articles/report/articles/category/26/27"]
for page in range(1,138):
    page_w_10.append("https://tfc-taiwan.org.tw/articles/report/articles/category/26/27?page=" + str(page))

reqs = (grequests.get(link) for link in page_w_10)  # 建立請求集合
response = grequests.imap(reqs, grequests.Pool(137))  # 發送請求

links = []
for r in response:
    
    soup = BeautifulSoup(r.content, "lxml") # 解析HTML原始碼
    a_tags = soup.find_all("div", {"class": "view-content"}) # 包含該頁10篇文章連結的區塊
    herfs = a_tags[0].find_all("a",attrs={'href': re.compile("^/articles/")}) # 取得區塊內的所有連結

    for link in herfs:
        if "category" in link.get('href') or link.get('href') in links: # 只保留事實查核報告的連結，且連結不重複
            continue
        links.append(link.get('href'))
len(links) # 共1373篇文章

/opt/anaconda3/lib/python3.7/site-packages/grequests.py:22: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.contrib.pyopenssl (/opt/anaconda3/lib/python3.7/site-packages/urllib3/contrib/pyopenssl.py)']. 
  curious_george.patch_all(thread=False, select=False)


1373

In [145]:
article_links = [] # 所有文章連結的 list
error_links = [] # 準備作為未成功爬取文章的連結 list
for i in links:
    article_links.append("https://tfc-taiwan.org.tw" + i)
    error_links.append("https://tfc-taiwan.org.tw" + i)
print(len(article_links))
print('===============')
print(len(error_links))

1373
1373


In [ ]:
print(error_links) 

In [ ]:
# DON'T MOVE
error_links_test= ['https://tfc-taiwan.org.tw/articles/6456', 'https://tfc-taiwan.org.tw/articles/6270', 'https://tfc-taiwan.org.tw/articles/6262', 'https://tfc-taiwan.org.tw/articles/6068', 'https://tfc-taiwan.org.tw/articles/6002', 'https://tfc-taiwan.org.tw/articles/6000', 'https://tfc-taiwan.org.tw/articles/5896', 'https://tfc-taiwan.org.tw/articles/5923', 'https://tfc-taiwan.org.tw/articles/5665', 'https://tfc-taiwan.org.tw/articles/5646', 'https://tfc-taiwan.org.tw/articles/5573', 'https://tfc-taiwan.org.tw/articles/5569', 'https://tfc-taiwan.org.tw/articles/5503', 'https://tfc-taiwan.org.tw/articles/5320', 'https://tfc-taiwan.org.tw/articles/5295', 'https://tfc-taiwan.org.tw/articles/5196', 'https://tfc-taiwan.org.tw/articles/5186', 'https://tfc-taiwan.org.tw/articles/5215', 'https://tfc-taiwan.org.tw/articles/5148', 'https://tfc-taiwan.org.tw/articles/5135', 'https://tfc-taiwan.org.tw/articles/5027', 'https://tfc-taiwan.org.tw/articles/5006', 'https://tfc-taiwan.org.tw/articles/4876', 'https://tfc-taiwan.org.tw/articles/4866', 'https://tfc-taiwan.org.tw/articles/4692', 'https://tfc-taiwan.org.tw/articles/4467', 'https://tfc-taiwan.org.tw/articles/4135', 'https://tfc-taiwan.org.tw/articles/3921', 'https://tfc-taiwan.org.tw/articles/3856', 'https://tfc-taiwan.org.tw/articles/3852', 'https://tfc-taiwan.org.tw/articles/3827', 'https://tfc-taiwan.org.tw/articles/3825', 'https://tfc-taiwan.org.tw/articles/3823', 'https://tfc-taiwan.org.tw/articles/3822', 'https://tfc-taiwan.org.tw/articles/3821', 'https://tfc-taiwan.org.tw/articles/3818', 'https://tfc-taiwan.org.tw/articles/3816', 'https://tfc-taiwan.org.tw/articles/3804', 'https://tfc-taiwan.org.tw/articles/3803', 'https://tfc-taiwan.org.tw/articles/3795', 'https://tfc-taiwan.org.tw/articles/3851', 'https://tfc-taiwan.org.tw/articles/3850', 'https://tfc-taiwan.org.tw/articles/3849', 'https://tfc-taiwan.org.tw/articles/3847', 'https://tfc-taiwan.org.tw/articles/3846', 'https://tfc-taiwan.org.tw/articles/3845', 'https://tfc-taiwan.org.tw/articles/3843', 'https://tfc-taiwan.org.tw/articles/3836', 'https://tfc-taiwan.org.tw/articles/3830', 'https://tfc-taiwan.org.tw/articles/3829', 'https://tfc-taiwan.org.tw/articles/3794', 'https://tfc-taiwan.org.tw/articles/3792', 'https://tfc-taiwan.org.tw/articles/3791', 'https://tfc-taiwan.org.tw/articles/3790', 'https://tfc-taiwan.org.tw/articles/3787', 'https://tfc-taiwan.org.tw/articles/3783', 'https://tfc-taiwan.org.tw/articles/3781', 'https://tfc-taiwan.org.tw/articles/3772', 'https://tfc-taiwan.org.tw/articles/3769', 'https://tfc-taiwan.org.tw/articles/3768', 'https://tfc-taiwan.org.tw/articles/3762', 'https://tfc-taiwan.org.tw/articles/3758', 'https://tfc-taiwan.org.tw/articles/3757', 'https://tfc-taiwan.org.tw/articles/3640', 'https://tfc-taiwan.org.tw/articles/2916', 'https://tfc-taiwan.org.tw/articles/2736', 'https://tfc-taiwan.org.tw/articles/2081', 'https://tfc-taiwan.org.tw/articles/2076', 'https://tfc-taiwan.org.tw/articles/1718', 'https://tfc-taiwan.org.tw/articles/1688', 'https://tfc-taiwan.org.tw/articles/919', 'https://tfc-taiwan.org.tw/articles/817', 'https://tfc-taiwan.org.tw/articles/636', 'https://tfc-taiwan.org.tw/articles/617', 'https://tfc-taiwan.org.tw/articles/616', 'https://tfc-taiwan.org.tw/articles/451', 'https://tfc-taiwan.org.tw/articles/486', 'https://tfc-taiwan.org.tw/articles/377', 'https://tfc-taiwan.org.tw/articles/355', 'https://tfc-taiwan.org.tw/articles/350', 'https://tfc-taiwan.org.tw/articles/349']
error_links_delete= ['https://tfc-taiwan.org.tw/articles/6456', 'https://tfc-taiwan.org.tw/articles/6270', 'https://tfc-taiwan.org.tw/articles/6262', 'https://tfc-taiwan.org.tw/articles/6068', 'https://tfc-taiwan.org.tw/articles/6002', 'https://tfc-taiwan.org.tw/articles/6000', 'https://tfc-taiwan.org.tw/articles/5896', 'https://tfc-taiwan.org.tw/articles/5923', 'https://tfc-taiwan.org.tw/articles/5665', 'https://tfc-taiwan.org.tw/articles/5646', 'https://tfc-taiwan.org.tw/articles/5573', 'https://tfc-taiwan.org.tw/articles/5569', 'https://tfc-taiwan.org.tw/articles/5503', 'https://tfc-taiwan.org.tw/articles/5320', 'https://tfc-taiwan.org.tw/articles/5295', 'https://tfc-taiwan.org.tw/articles/5196', 'https://tfc-taiwan.org.tw/articles/5186', 'https://tfc-taiwan.org.tw/articles/5215', 'https://tfc-taiwan.org.tw/articles/5148', 'https://tfc-taiwan.org.tw/articles/5135', 'https://tfc-taiwan.org.tw/articles/5027', 'https://tfc-taiwan.org.tw/articles/5006', 'https://tfc-taiwan.org.tw/articles/4876', 'https://tfc-taiwan.org.tw/articles/4866', 'https://tfc-taiwan.org.tw/articles/4692', 'https://tfc-taiwan.org.tw/articles/4467', 'https://tfc-taiwan.org.tw/articles/4135', 'https://tfc-taiwan.org.tw/articles/3921', 'https://tfc-taiwan.org.tw/articles/3856', 'https://tfc-taiwan.org.tw/articles/3852', 'https://tfc-taiwan.org.tw/articles/3827', 'https://tfc-taiwan.org.tw/articles/3825', 'https://tfc-taiwan.org.tw/articles/3823', 'https://tfc-taiwan.org.tw/articles/3822', 'https://tfc-taiwan.org.tw/articles/3821', 'https://tfc-taiwan.org.tw/articles/3818', 'https://tfc-taiwan.org.tw/articles/3816', 'https://tfc-taiwan.org.tw/articles/3804', 'https://tfc-taiwan.org.tw/articles/3803', 'https://tfc-taiwan.org.tw/articles/3795', 'https://tfc-taiwan.org.tw/articles/3851', 'https://tfc-taiwan.org.tw/articles/3850', 'https://tfc-taiwan.org.tw/articles/3849', 'https://tfc-taiwan.org.tw/articles/3847', 'https://tfc-taiwan.org.tw/articles/3846', 'https://tfc-taiwan.org.tw/articles/3845', 'https://tfc-taiwan.org.tw/articles/3843', 'https://tfc-taiwan.org.tw/articles/3836', 'https://tfc-taiwan.org.tw/articles/3830', 'https://tfc-taiwan.org.tw/articles/3829', 'https://tfc-taiwan.org.tw/articles/3794', 'https://tfc-taiwan.org.tw/articles/3792', 'https://tfc-taiwan.org.tw/articles/3791', 'https://tfc-taiwan.org.tw/articles/3790', 'https://tfc-taiwan.org.tw/articles/3787', 'https://tfc-taiwan.org.tw/articles/3783', 'https://tfc-taiwan.org.tw/articles/3781', 'https://tfc-taiwan.org.tw/articles/3772', 'https://tfc-taiwan.org.tw/articles/3769', 'https://tfc-taiwan.org.tw/articles/3768', 'https://tfc-taiwan.org.tw/articles/3762', 'https://tfc-taiwan.org.tw/articles/3758', 'https://tfc-taiwan.org.tw/articles/3757', 'https://tfc-taiwan.org.tw/articles/3640', 'https://tfc-taiwan.org.tw/articles/2916', 'https://tfc-taiwan.org.tw/articles/2736', 'https://tfc-taiwan.org.tw/articles/2081', 'https://tfc-taiwan.org.tw/articles/2076', 'https://tfc-taiwan.org.tw/articles/1718', 'https://tfc-taiwan.org.tw/articles/1688', 'https://tfc-taiwan.org.tw/articles/919', 'https://tfc-taiwan.org.tw/articles/817', 'https://tfc-taiwan.org.tw/articles/636', 'https://tfc-taiwan.org.tw/articles/617', 'https://tfc-taiwan.org.tw/articles/616', 'https://tfc-taiwan.org.tw/articles/451', 'https://tfc-taiwan.org.tw/articles/486', 'https://tfc-taiwan.org.tw/articles/377', 'https://tfc-taiwan.org.tw/articles/355', 'https://tfc-taiwan.org.tw/articles/350', 'https://tfc-taiwan.org.tw/articles/349']


## 建立資料集

In [188]:
# for loop

# 製作爬取兩個tag間文字的function
#     def between(cur, end):
#         while cur and cur != end:
#             if isinstance(cur, NavigableString):
#                 text = cur.strip()
#                 if len(text):
#                     yield text
#             cur = cur.next_element

start_time = time.time()

count = -1
count_all = -1
id_list = [] # id
text_list = [] # 訊息文本
result_list = [] # 查核結果
subject_list = [] # 主題分類
date_list = [] # 更新日期
link_list = [] # 查核文章連結

for link in article_links[900:1373]:
    
    count_all+=1

    # 連結網站
    response = requests.get(link)

    # HTML原始碼解析
    soup = BeautifulSoup(response.text, "html.parser")

    # 輸出訊息文本（介於'背景'與'查核'之間的文字）without .next_sibling
    tem_article = ' '.join(text for text in between(soup.find('h2', text=re.compile('背景')),soup.find('h2', text=re.compile('查核'))))
    if len(tem_article)!=0:
        article = re.search('「(.*)」', tem_article)
        if article != None:
            try:
                text_list.append(article.group(1))
            except AttributeError:
                text_list.append('null')
        elif re.search('稱：(.*)圖1',tem_article) != None:
            article2 = re.search('稱：(.*)圖1',tem_article)
            try:
                text_list.append(article2.group(1))
            except AttributeError:
                text_list.append('null')
        else:
            tem_article2 = ' '.join(text for text in between(soup.find('p', text=re.compile('背景')),soup.find('h2', text=re.compile('查核'))))
            article = re.search('：(.*)圖1', tem_article2)
            try:
                text_list.append(article.group(1))
            except AttributeError:
                text_list.append('null')
    else:
        text_list.append('null')

    # 取得查核結果與主題分類
    node_tags = soup.find_all("div", {"class": "node-tags"})
    node_tag = node_tags[0].get_text()
    node_tag_filter = list(filter(None, node_tag.split("\n")))

    try:
        result_list.append(node_tag_filter[1]) # 查核結果
    except IndexError:
        result_list.append('null')

    try:
        subject_list.append(node_tag_filter[2]) # 主題分類
    except IndexError:
        subject_list.append('null')

    # 取得更新日期
    submitted = soup.find_all("div", {"class": "submitted"})
    date = datetime.strptime(submitted[0].get_text(), "%Y-%m-%d").date()
    date_list.append(date)

    # 其他計算
    count+=1
    id_list.append(count)
    link_list.append(link)

print("--- %s seconds ---" % (time.time() - start_time))


--- 460.7077121734619 seconds ---


In [189]:
print(len(id_list))
print(len(text_list))
print(len(result_list))
print(len(subject_list))
print(len(date_list))
print(len(link_list))
print(len(article_links)-len(link_list)-len(link_list)-len(link_list)-len(link_list))
print(len(error_links)-len(link_list)-len(link_list))
print('=============')
print(count_all)
print(count)

473
473
473
473
473
473
-519
-173
472
472


In [187]:
article_links[1372]

'https://tfc-taiwan.org.tw/articles/84'

In [190]:
d = {'id': id_list, 'text': text_list, 'result': result_list, 'subject': subject_list, 'date': date_list, 'link': link_list}
df = pd.DataFrame(data=d)
df
df.to_csv('factcheck_900_1373.csv', index=False, encoding='utf-8')

## 合併四個csv檔案

In [191]:
# importing pandas
import pandas as pd
  
# merging two csv files
df = pd.concat(
    map(pd.read_csv, ['factcheck_0_300.csv', 'factcheck_300_600.csv', 'factcheck_600_900.csv', 'factcheck_900_1373.csv']), ignore_index=True)

,id,text,result,subject,date,link
0,0,108 導播室」12月3日發布一則 標題「高嘉瑜酒店不雅視頻流出?」影片，內容從4分開始到1...,錯誤,政治與政策,2021-12-06,https://tfc-taiwan.org.tw/articles/6643
1,1,最新流出，立委高嘉瑜不雅影片，看完請立即刪除，切勿流傳以免觸法。,錯誤,生活,2021-12-03,https://tfc-taiwan.org.tw/articles/6637
2,2,12月2日！拜登最害怕的事发生了！美国民主峰会彻底搞砸了！110国元首当场退席！中美地位一夜...,錯誤,國際,2021-12-03,https://tfc-taiwan.org.tw/articles/6636
3,3,一定要戴上口罩，因為新的 COVID-Omicron 變種冠狀病毒是不同的，致命的和不容易正...,錯誤,健康,2021-12-02,https://tfc-taiwan.org.tw/articles/6634
4,4,你平時戴口罩是不是撕開這個封膜就開始往嘴上戴，那我告訴你，這個做法是錯誤的，為什麼撕開封膜就...,錯誤,健康,2021-12-01,https://tfc-taiwan.org.tw/articles/6633
...,...,...,...,...,...,...
1368,468,法國世足賽奪得冠軍，熱烈歡慶中，衷心銘感台灣製作好穿的衣褲，感激大功勞之餘，刻意安排出台灣的...,錯誤,國際,2018-08-03,https://tfc-taiwan.org.tw/articles/100
1369,469,以下是澎湖縣體育會總幹事發表的內容：大家別在被執政者騙了，又在移花接木，明明是自己私自更改運...,錯誤,政治與政策,2018-08-03,https://tfc-taiwan.org.tw/articles/98
1370,470,不用我來說，誰都可以看出來背後老闆是誰，也都猜得出來那些人在玩這種幽靈航空公司的把戲。」該傳...,錯誤,政治與政策,2018-07-31,https://tfc-taiwan.org.tw/articles/83
1371,471,堅不與我斷交還無懼旅遊禁令 帛琉航空停飛中國」，「無懼陸恐嚇堅持不與台灣斷交 帛琉航空停飛大...,錯誤,政治與政策,2018-07-31,https://tfc-taiwan.org.tw/articles/79


In [204]:
for_id = []
for i in df['link']:
    for_id.append(i.replace('https://tfc-taiwan.org.tw/articles/',''))
df['id'] = for_id
df

,id,text,result,subject,date,link
0,6643,108 導播室」12月3日發布一則 標題「高嘉瑜酒店不雅視頻流出?」影片，內容從4分開始到1...,錯誤,政治與政策,2021-12-06,https://tfc-taiwan.org.tw/articles/6643
1,6637,最新流出，立委高嘉瑜不雅影片，看完請立即刪除，切勿流傳以免觸法。,錯誤,生活,2021-12-03,https://tfc-taiwan.org.tw/articles/6637
2,6636,12月2日！拜登最害怕的事发生了！美国民主峰会彻底搞砸了！110国元首当场退席！中美地位一夜...,錯誤,國際,2021-12-03,https://tfc-taiwan.org.tw/articles/6636
3,6634,一定要戴上口罩，因為新的 COVID-Omicron 變種冠狀病毒是不同的，致命的和不容易正...,錯誤,健康,2021-12-02,https://tfc-taiwan.org.tw/articles/6634
4,6633,你平時戴口罩是不是撕開這個封膜就開始往嘴上戴，那我告訴你，這個做法是錯誤的，為什麼撕開封膜就...,錯誤,健康,2021-12-01,https://tfc-taiwan.org.tw/articles/6633
...,...,...,...,...,...,...
1368,100,法國世足賽奪得冠軍，熱烈歡慶中，衷心銘感台灣製作好穿的衣褲，感激大功勞之餘，刻意安排出台灣的...,錯誤,國際,2018-08-03,https://tfc-taiwan.org.tw/articles/100
1369,98,以下是澎湖縣體育會總幹事發表的內容：大家別在被執政者騙了，又在移花接木，明明是自己私自更改運...,錯誤,政治與政策,2018-08-03,https://tfc-taiwan.org.tw/articles/98
1370,83,不用我來說，誰都可以看出來背後老闆是誰，也都猜得出來那些人在玩這種幽靈航空公司的把戲。」該傳...,錯誤,政治與政策,2018-07-31,https://tfc-taiwan.org.tw/articles/83
1371,79,堅不與我斷交還無懼旅遊禁令 帛琉航空停飛中國」，「無懼陸恐嚇堅持不與台灣斷交 帛琉航空停飛大...,錯誤,政治與政策,2018-07-31,https://tfc-taiwan.org.tw/articles/79


In [205]:
df.to_csv('factcheck.csv', index=False, encoding='utf-8')

# 延伸應用 關鍵字搜尋

In [262]:
all_text = []
for i in range(0,1373):
    all_text.append(str(df['text'][i]))

In [ ]:
while True:
    check_str = str(input('輸入想查核的訊息：'))
    print('')
    matching = [x for x in all_text if check_str in x]
    if len(matching)!=0:
        print('與「',check_str,'」相關的訊息有',len(matching),'則：','\n',sep='')
        check_index = []
        for i in range(0,len(matching)):
            check_index.append(all_text.index(str(matching[i])))
            print('編號',check_index[i],'. ','\n',all_text[check_index[i]],'\n',sep='')
        check = int(input('請輸入您要查核的訊息編號：'))
        if check in check_index:
            print('查核結果為「',df['result'][check],'」消息',sep='')
            print('查核相關資料請見',df['link'][check],'\n')
            cont = int(input('是否要繼續查核其他訊息？（1:是, 0:否）'))
            if cont == 1:
                continue
            else:
                break
        else:
            cont = int(input('查無結果，是否要繼續查核其他訊息？（1:是, 0:否）'))
            if cont == 1:
                continue
            else:
                break
    else:
        cont = int(input('資料庫無相關訊息可供查證，是否要繼續查核其他訊息？（1:是, 0:否）'))
        if cont == 1:
                continue
        else:
            break
print('感謝您使用本查核系統')